<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Multiple_pdf_chat_using_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain PyPDF2 openai tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.0 MB/s eta 0:00:00


In [2]:
! pip list --local

Package                       Version
----------------------------- ---------------------
absl-py                       1.4.0
aiohttp                       3.8.4
aiosignal                     1.3.1
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
array-record                  0.2.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
async-timeout                 4.0.2
attrs                         23.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
branc

In [3]:
! pip install -q huggingface-hub==0.14.1
! pip install -q InstructorEmbedding==1.0.1
! pip install -q sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.6 MB/s eta 0:00:00


In [4]:
import openai
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# from htmlTemplates import css, bot_template, user_template
from langchain.llms import HuggingFaceHub
from getpass import getpass
import os
import json

In [5]:
os.environ["OPENAI_API_KEY"] = getpass()

··········


In [26]:
#
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks


def get_vectorstore(text_chunks,api_key):
    embeddings = OpenAIEmbeddings(openai_api_key=api_key)
    # embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore


def get_conversation_chain(vectorstore,api_key):
    llm = ChatOpenAI(openai_api_key=api_key)
    # llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm, retriever=vectorstore.as_retriever(), memory=memory
    )
    return conversation_chain

#



In [17]:
os.mkdir("model_files")

In [22]:
#
from configparser import ConfigParser
#
ai_for_world_config = ConfigParser()
#
def read_config(parser: ConfigParser, location: str) -> None:
    assert parser.read(location), f"Could not read config {location}"

In [42]:
def preprocess_function(file_path,prompt,content_type=None):
    '''
    # Create a temporary file.
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:

        tmp_file.write(read_inputfile(file_path))  # write the data into the temporary file
    file_name = tmp_file.name
    pdf_loader = PyPDFLoader(file_name)
    #ppt_loader = UnstructuredPowerPointLoader(file_path)
    pdf = pdf_loader.load()
    print(f"Number of pages in the document {len(pdf)}")
    '''
    text = ""
    pdf_docs = os.listdir(file_path)
    for pdf in pdf_docs:
        pdf_path = os.path.join(file_path,pdf)
        pdf_reader = PdfReader(pdf_path)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return (text,prompt)
#
def model_load_function(model_path):
    CONFIG_FILE = os.path.join(model_path, "env.conf")
    read_config(ai_for_world_config, CONFIG_FILE)
    api_key = ai_for_world_config.get("openai", "api_key").strip()
    return api_key


#

def predict_function(context, api_key):
    raw_text,prompt = context
    print(prompt)
    # get the text chunks
    text_chunks = get_text_chunks(raw_text)

    # create vector store
    vectorstore = get_vectorstore(text_chunks,api_key)

    # create conversation chain
    conversation = get_conversation_chain(vectorstore,api_key)
    #
    response = conversation.run(prompt)
    return response
#

def postprocess_function(predictions, content_type=None):
  return json.dumps({"response": predictions})


## Test the script
"""
if __name__ == '__main__':
    audio_path = "chatgpt prompts.pdf"
    data= preprocess_function(audio_path)
    model_path = "./model_files"
    path = model_load_function(model_path)
    predictions = predict_function(data,path)
    out = postprocess_function(predictions)
    print(out)
"""

'\nif __name__ == \'__main__\':\n    audio_path = "chatgpt prompts.pdf"\n    data= preprocess_function(audio_path)\n    model_path = "./model_files"\n    path = model_load_function(model_path)\n    predictions = predict_function(data,path)\n    out = postprocess_function(predictions)\n    print(out)\n'

In [33]:
file_path = "/content/pdf_path"
prompt = "How to push a file to github"
context = preprocess_function(file_path,prompt,content_type=None)

In [34]:
text,prompt = context
prompt

'How to push a file to github'

In [35]:
model_path = "/content/model_files"
api_key = model_load_function(model_path)

In [38]:
predictions = predict_function(context,api_key)

How to push a file to github


In [40]:
print(predictions)

To push a file to Github, you need to follow these steps:

1. First, navigate to the local directory where the file is located using the command line or terminal.
2. Use the command "git add filename" to add the file to the staging area. You can also use "git add ." to add all the files in the directory.
3. Use the command "git commit -m 'commit message'" to commit the changes with a descriptive message.
4. Finally, use the command "git push" to push the changes to the Github repository.

Make sure you have the necessary permissions to push changes to the repository.


In [43]:
print(postprocess_function(predictions))

{"response": "To push a file to Github, you need to follow these steps:\n\n1. First, navigate to the local directory where the file is located using the command line or terminal.\n2. Use the command \"git add filename\" to add the file to the staging area. You can also use \"git add .\" to add all the files in the directory.\n3. Use the command \"git commit -m 'commit message'\" to commit the changes with a descriptive message.\n4. Finally, use the command \"git push\" to push the changes to the Github repository.\n\nMake sure you have the necessary permissions to push changes to the repository."}
